In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using NearestNeighbors

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [145]:
import BlossomV: Matching, add_edge, get_match

Matching(node_num::Integer) = Matching(node_num, node_num*(node_num-1)÷2)      
Matching(node_num::Integer, edge_num_max::Integer) = Matching(Int32(node_num), Int32(edge_num_max))

add_edge(matching::BlossomV.PerfectMatchingCtx,
        first_node::Integer,
        second_node::Integer, cost::Integer) = add_edge(matching, 
                                                        Int32(first_node), 
                                                        Int32(second_node),
                                                        Int32(cost))

get_match(matching::BlossomV.PerfectMatchingCtx, node::Integer) = get_match(matching, Int32(node))


"1 indexed" #TODO make this zero indexed
function get_all_matchs(m::PerfectMatchingCtx, n_nodes::Integer) #HACK, you have to specidfiy the number of nodes, cos I can't pull it from the Matching object
    Task() do
        assigned = falses(n_nodes)
        for ii in 1:n_nodes
            assigned[ii] && continue
            jj = get_match(m, ii-1)+1
            @assert(!assigned[jj])
            assigned[ii] = true
            assigned[jj] = true
            
            produce(ii,jj)
        end
    end
end




get_all_matchs

In [2]:
using JLD


In [3]:
ee = load("../eval/models/plain/tokenised_lowercase_WestburyLab.wikicorp.201004_50__i1.jld","ee");
#ee = restore("../eval/models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model")
#dtree,labels = nn_tree(ee)
#""

Word embedding(dimension = 50)of 88262 words


In [4]:
#ee.classification_tree.children[1]

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [5]:
using Trees

In [8]:
words = collect(keys(ee.codebook))
codes = collect(values(ee.codebook));


In [109]:
function get_node(root::TreeNode, pathcode)
    node = root
    for branch_step in pathcode
        node = node.children[branch_step]using BlossomV

    node     
end

LoadError: LoadError: syntax: extra token "using" after end of expression
while loading In[109], in expression starting on line 4

In [15]:
getMatching_node(ee.classification_tree, codes[1])

"rearrangement"

Trees.BranchNode with 0 children.data = 


In [28]:
function Base.show(io::IO, node :: BranchNode)                                                                       
    println(typeof(node)," with ",length(node.children), " children. ", "data = ")                                   
    flush(io)
    show(io, node.data)                                                                                              
end          
BranchNode([], "foobar")

"foobar"

Trees.BranchNode with 0 children.data = 


In [39]:
function levels(tree)
    nodes_at_depth = Vector{Vector{BranchNode}}()
    codes_at_depth = Vector{Vector{Vector{Int64}}}()
    
    push!(nodes_at_depth, tree.children) #TODO: fix so works for partiailly empty root node
    push!(codes_at_depth, [[ii] for ii in 1:length(tree.children)])
    
    
    
    parent_depth = 1
    while length(nodes_at_depth[parent_depth])>0
        #@show parent_depth
        level_nodes = BranchNode[]
        level_codes = Vector{Int64}[]
        
        

        for (parent_code, parent_node) in zip(codes_at_depth[parent_depth], nodes_at_depth[parent_depth])
            #if Trees.isleaf(parent_node)
            #    continue
            #end
            
            for (ii, child_node) in enumerate(parent_node.children)
                if child_node != nullnode
                    child_code = [parent_code; ii] #Copy and append
                    push!(level_nodes, child_node)
                    push!(level_codes, child_code)
                end
            end
        end
        push!(nodes_at_depth, level_nodes)
        push!(codes_at_depth, level_codes)
        parent_depth += 1
    end
    
    nodes_at_depth, codes_at_depth
end


levels (generic function with 1 method)

In [40]:
nodes_at_depth, codes_at_depth = levels(ee.classification_tree);

In [42]:
"Returns a new tree, with the same structure but different values for the data"
function map_treedata(node::BranchNode; leaf_transform=identity, internal_transform=identity)
    data = Tree.isleaf(node) ? leaf_transform(node.data) : internal_transform(node.data)
    children = BranchNode[map_treedata(child, leaf_transform, internal_transform)  for child in node.children]
    new_node = BranchNode(copy(node.children), data)
end
    

map_treedata

In [26]:
"""
Assumes that the `tree` is already a Huffman tree.
"""
function semhuff(classification_tree, embeddings)
    embedding_dim = length(first(embeddings))
    sim_tree = map_treedata(classification_tree,
                            leaf_transform = word->embeddings[word],
                            internal_transform = dummy -> fill(NaN32,embedding_dim) )
    
    nodes_at_depth, codes_at_depth = levels(sim_tree);
    maxdepth = length(nodes_at_depth)
    for depth in maxdepth:-1:1
        
    end
    
end
    
    

Trees.BranchNode

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [112]:
"""dist_func must resturn a values between 0.0 and 1.0"""
function get_similar_pairings(dist_func::Function, items, consider_nearest_n::Integer)
    m = Matching(length(items))
    sims = Matrix{Int32}(length(items),length(items)).*Inf #for Debug purposes
    for ii::Int32 in (1:length(items)-1)
        dists = [dist_func(items[ii], items[jj]) for jj::Int32 in (ii+1:length(items))]
        nearest_jjs = selectperm(dists, 1:consider_nearest_n)
        
        for jj in nearest_jjs
            dist = dists[jj]
            @assert dist<=1.0
            scale = typemax(Int32)>>4
            approx_dist = round(Int32, dist*scale)
            sims[ii,jj]=approx_dist
            #println(join([m, ii-1, jj-1, approx_dist],"\t"))
            add_edge(m, ii-1,jj-1 , approx_dist)
        end
    end
    sovle(m)
    get_all_matchs(m, length(items)), sims
    
end

LoadError: LoadError: syntax: unexpected ,
while loading In[112], in expression starting on line 20

In [46]:
ee_keys = rand(1:length(ee.embedding),6)
wvs = collect(values(ee.embedding))[ee_keys]
words = collect(keys(ee.embedding))[ee_keys]

6-element Array{String,1}:
 "statistical"  
 "otherworld"   
 "428"          
 "edie"         
 "airspeed"     
 "light-hearted"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


get_match (generic function with 2 methods)

In [139]:
using Query
wvs = [randn(50) for i in 1:6]
m=Matching(Int32(length(wvs)),Int32(length(wvs))^2)

BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)

In [140]:
sims = Matrix{Int32}(length(wvs),length(wvs)).*Inf
for ii::Int32 in (1:length(wvs)-1)
    wv_ii=wvs[ii]
    for jj::Int32 in (ii+1:length(wvs))
        
        wv_jj=wvs[jj]
        dist = Query.angular_dist(wv_ii, wv_jj)
        
        scale = typemax(Int32)>>16
        approx_dist = round(Int32, dist*scale)
        sims[ii,jj]=approx_dist
        println(join([m, ii-1, jj-1, approx_dist],"\t"))
        add_edge(m, ii-1,jj-1 , approx_dist)
    end
end
sims

BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	0	1	14834
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	0	2	15856
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	0	3	15635
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	0	4	13802
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	0	5	17562
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	1	2	14384
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	1	3	14517
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	1	4	16561
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	1	5	16676
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	2	3	19132
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	2	4	17143
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	2	5	17955
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	3	4	17518
BlossomV.PerfectMatchingCtx(Ptr{Void} @0x000000001176c9d0)	3	5	18466
BlossomV.PerfectMatchingCtx(Ptr{Vo

6×6 Array{Float64,2}:
 Inf  14834.0  15856.0  15635.0  13802.0  17562.0
 Inf    Inf    14384.0  14517.0  16561.0  16676.0
 Inf    Inf      Inf    19132.0  17143.0  17955.0
 Inf    Inf      Inf      Inf    17518.0  18466.0
 Inf    Inf      Inf      Inf      Inf    18091.0
 Inf    Inf      Inf      Inf      Inf      Inf  

In [141]:
BlossomV.solve(m)

perfect matching with 6 nodes and 15 edges
    starting init...done [0.000 secs]. 0 trees
    .
done [0.000 secs]. 0 grows, 0 expands, 0 shrinks
    expands: [0.000 secs], shrinks: [0.000 secs], dual updates: [0.000 secs]


In [142]:
for ii in (1:6)-1
    match = get_match(m,ii)+1
    greed = indmin(sims[ii+1,:]-1)
    println(ii+1,"\t|\t", match, "\t", greed)
end

1	|	5	5
2	|	4	3
3	|	6	5
4	|	2	5
5	|	1	6
6	|	3	1


In [143]:

collect(get_all_matchs(m,6))

3-element Array{Tuple{Int64,Int64},1}:
 (1,5)
 (2,4)
 (3,6)

In [132]:
typeof(m)

BlossomV.PerfectMatchingCtx

In [ ]:
@inline function Base.exp2(x::Int64)
    if x > 1023 
        Inf 
    elseif x < -1074
        0.0
    else 
        reinterpret(Float64,x > -1023 ? ((1<<62)+(x-1)<<52) : 1<<(x+1074))
    end
end

function get_path_dist(code1,code2)
    l1 = length(code1)
    l2 = length(code2)
    lc = 0 #Common length
    for lc in 1:min(l1,l2)
        if code1[lc]!=code2[lc]
            lc-=1 #Last didn't match
            break
        end
    end
    ld1 = l1-lc
    ld2 = l2-lc
    ld1+ld2
end

get_path_sim(code1,code2) = exp2(-get_path_dist(code1,code2))

using Base.Test
@test get_path_sim([1],[1]) == 1.0
@test get_path_sim([1],[0]) == 0.25

@test get_path_sim([1,1],[1,0]) == 0.25
@test get_path_sim([1,1],[1,1]) == 1.0
@test get_path_sim([1,1],[0,1]) == 1/16
@test get_path_sim([1,1],[0,0]) == 1/16

@test get_path_sim([1],[1,1]) == 0.5
@test get_path_sim([1],[1,1,1]) == 0.25
@test get_path_sim([1,1],[1,1,1]) == 0.5
@test get_path_sim([1,0],[1,1,1]) == 0.125
@test get_path_sim([1,1,1],[1,1,1]) == 1.0


In [ ]:
ee 